# Notes about `melspectrogram.tflite`

* Wygląda, że `melspectrogram.tflite` jest bez potrzeby zbyt duży.
* Można policzyć spektogram mel za pomocą FFT, a następnie mnorząc przez macierz utworzoną przez `librosa.filters.mel`.
  Macierz ma niezerowe wartości głównie koło diagonali, więc można mnożenie mocno zoptymalizować.
  Przez FFT jest kilka kilo operacji, a modelem jest kilkaset K operacji.
* Wygląda, że też nie do końca działa identycznie z oryginalnym modelem z https://www.kaggle.com/models/google/speech-embedding/tensorFlow1/speech-embedding/1.
  Dla sinusoidy jest nieco rozmyty i nie jestem pewien czy skala mel jest w 100% poprawna.
* Oryginalny model brał okno o rozmiarze 400 sampli (2.5 skoku), a `melspectrogram.tflite` bierze 480 sampli (3 skoki)
* Jeżeli będzie trzeba pisać FFT ręcznie (albo przynajmniej mel z FFT):
  * Etapy są następujące:
    * Jeżeli trzeba to, dodać padding: 400 -> 512 = 112 (chociaż liczenie FFT 512 powinno dać bardzo zbliżone rezultaty)
    * RFFT (real FFT):
      * scalić dwie kolejne liczby rzeczywiste w jedną liczbę zespoloną (256 liczb zespolonych będzie razem)
      * przetasować wejście
      * przelecieć motylkami log(N) razy
      * wyciągnąć pierwszą połowę spektrum https://chatgpt.com/share/6859b9e7-30d0-8012-8919-6093422e9029
      * Teoretycznie, jeżeli przesunąc przetasowanie do wyjścia, to można wykożystać dane policzone w poprzednim oknie,
        ale to tylko dla pierwszego etapu motylków i małej liczby wartości. Ostatecznie, to się nie opłaca.
    * Policzyć amplitudę (ComplexAbs)
    * Pomnożyć prze macierz zwróconą przez `librosa.filters.mel` (zoptymalizowane mnożenie tylko nie zerowych wartości)
    * Dodać 0.001
    * Policzyć logarytm


### Poniżej jest kilka przykładów i ekperymentów na temat powyższych rzeczy

In [ ]:
import sys
import os
import numpy as np
import onnxruntime as ort
import matplotlib.pyplot as plt
import torch
from torchaudio.transforms import MelSpectrogram
import src.melspec as melspec

# 📐 Parameters
F = 1495        # Frequency in Hz (e.g., A4)
Fs = 16000     # Sample rate
N = Fs * 2     # Duration in samples (2 seconds)

# 🎵 Generate sine wave
t = np.arange(N) / Fs
sine_wave = np.sin(2 * np.pi * F * t).astype(np.float32)
sine_wave = sine_wave[None, :]  # Add batch dimension [1, N]

# 📤 Load ONNX model
model_path = "models/melspectrogram.onnx"
session = ort.InferenceSession(model_path)
input_name = session.get_inputs()[0].name

# ⚙️ Run inference
output = session.run(None, {input_name: sine_wave})[0]  # Shape: [1, T, 32]
output = output.squeeze(0).T  # Transpose to [32 (mel bins), time frames]
print(output.shape)  # Should be [32, T]

# 🎨 Plot mel spectrogram
plt.figure(figsize=(10, 4))
plt.imshow(output, origin='lower', aspect='auto', cmap='magma',
           extent=[0, output.shape[1] * (160 / Fs), 0, 32])
plt.title(f'Mel Spectrogram of {F}Hz Sine Wave')
plt.xlabel('Time (s)')
plt.ylabel('Mel Bin Index')
plt.colorbar(label='Amplitude')
# Set magnitude range from -20 dB to 20 dB
plt.clim(-10, 30)
plt.tight_layout()
plt.show()

mel = MelSpectrogram(
    sample_rate=Fs,
    n_fft=400,
    win_length=400,
    hop_length=160,
    f_min=60,
    f_max=3800,
    n_mels=32,
    )

output = mel(torch.tensor(sine_wave))
output = 10 * 0.4342944622039795 * torch.log(output + 1e-3)  # Convert to dB
output = output.squeeze(0)
#output = np.log(output)  # Logarithmic scaling
print(output.shape)  # Should be [32, T]

plt.figure(figsize=(10, 4))
plt.imshow(output, origin='lower', aspect='auto', cmap='magma',
           extent=[0, output.shape[1] * (160 / Fs), 0, 32])
plt.title(f'Mel Spectrogram of {F}Hz Sine Wave')
plt.xlabel('Time (s)')
plt.ylabel('Mel Bin Index')
plt.colorbar(label='Amplitude')
# Set magnitude range from -20 dB to 20 dB
plt.clim(-10, 30)
plt.tight_layout()
plt.show()


mel = melspec.create_mel_spectrogram()

output = mel(torch.tensor(sine_wave))
output = 10 * 0.4342944622039795 * torch.log(output + 1e-3)  # Convert to dB
output = output.squeeze(0)
#output = np.log(output)  # Logarithmic scaling
print(output.shape)  # Should be [32, T]

plt.figure(figsize=(10, 4))
plt.imshow(output, origin='lower', aspect='auto', cmap='magma',
           extent=[0, output.shape[1] * (160 / Fs), 0, 32])
plt.title(f'Mel Spectrogram of {F}Hz Sine Wave')
plt.xlabel('Time (s)')
plt.ylabel('Mel Bin Index')
plt.colorbar(label='Amplitude')
# Set magnitude range from -20 dB to 20 dB
plt.clim(-10, 30)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np

def hz_to_mel(f):
    return 2595 * np.log10(1 + f / 700)

def mel_to_hz(m):
    return 700 * (10**(m / 2595) - 1)

# Frequency bounds
f_min = 60
f_max = 3800
n_mels = 32

# Step 1: Convert bounds to mel
mel_min = hz_to_mel(f_min)
mel_max = hz_to_mel(f_max)

# Step 2: Generate 32 linearly spaced points in mel scale
mel_points = np.linspace(mel_min, mel_max, n_mels)

# Step 3: Convert mel points back to Hz
center_frequencies_hz = mel_to_hz(mel_points)

# Print nicely
for i, f in enumerate(center_frequencies_hz):
    print(f"Mel bin {i+1:2d}: {f:.2f} Hz")


In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

#loaded = hub.load("https://www.kaggle.com/models/google/speech-embedding/TensorFlow1/speech-embedding/1")
loaded = hub.load('https://tfhub.dev/google/speech_embedding/1')
mod = loaded.signatures["default"]
#mod2 = loaded2.signatures["default"]



In [ ]:
# Inspect the model

logdir = "./logdir"
writer = tf.summary.create_file_writer(logdir)
with writer.as_default():
    tf.summary.graph(mod.graph)
writer.close()

# RUN IN TERMINAL:
# tensorboard --logdir=./logdir


In [ ]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([mod])
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # TFLite native ops
    tf.lite.OpsSet.SELECT_TF_OPS     # Include TF ops in TFLite
]
tflite_model = converter.convert()
with open("models/conv_speech_embedding.tflite", "wb") as f:
    f.write(tflite_model)
tf.saved_model.save(loaded, "models/conv_speech_embedding")


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tf2onnx

# Load model
hub_model = hub.load("https://tfhub.dev/google/speech_embedding/1")
default_fn = hub_model.signatures["default"]
print(default_fn.structured_input_signature)

input_spec = [tf.TensorSpec([None, 16000], tf.float32, name="default")]

@tf.function(input_signature=input_spec)
def model_fn(x):
    # Pass argument with correct name 'default'
    return default_fn(default=x)

onnx_model, _ = tf2onnx.convert.from_function(
    function=model_fn,
    input_signature=input_spec,
    opset=13,
    output_path="speech_embedding.onnx"
)


In [ ]:
# Inspect the model

melspec_model = tf.lite.Interpreter(model_path=str('models/melspectrogram.tflite'))
melspec_model.resize_tensor_input(
    melspec_model.get_input_details()[0]['index'],
    [1, 10000],
    strict=True)
melspec_model.allocate_tensors()
writer = tf.summary.create_file_writer('./logdir')
with writer.as_default():
    tf.summary.graph(melspec_model.graph)
writer.close()

# RUN IN TERMINAL:
# tensorboard --logdir=./logdir


In [ ]:
import librosa
mel_filterbank = librosa.filters.mel(sr=Fs, n_fft=512, n_mels=32, fmin=f_min, fmax=f_max)
